# Week 2: Language Modeling I

In this week's section, we'll explore basic techniques for language modeling. 

We'll first dig into word distributions to illustrate the sparsity problem. Then, we'll see how to build an N-gram language model, how to generate sample sentences, and how to use smoothing to improve performance.

We'll use [Plotly](https://plot.ly) for plotting some data in this notebook. They're a commercial service, but the plotting library is open-source and provides a much more modern, high-level interface than `matplotlib`. Install it with:
```
pip install plotly
```
You can check out their documentation [here](https://plot.ly/python/), but the code in this notebook should be self-explanatory if you're familiar with `matplotlib`, `ggplot`, or `bokeh`.

If you get an import error below, go to **Kernel -> Restart** after running the `pip` command.

In [ ]:
#!pip install --upgrade plotly
#!pip install --upgrade nltk
import os, sys, re, json, time
import itertools
import collections

import numpy as np

import nltk
nltk.download()
# Helper libraries for this notebook
import data_utils
import vocabulary

# Plotly
import plotly.offline as plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
       |   Unzipping corpora/cess_esp.zip.
       | Downloa

## Corpus Statistics

NLTK includes a number of corpora that we can experiment with for this exercise. Different types of text can have very different N-gram distributions, and some are more difficult to model than others.

Let's start with the [Brown corpus](http://www.essex.ac.uk/linguistics/external/clmt/w3c/corpus_ling/content/corpora/list/private/brown/brown.html), the first major computer-readable linguistic corpus. It consists of around 1 million words of American English, sampled from 15 different text categories ranging from news text to academic articles to popular fiction.

**Note:** If you get an error loading this or any other corpus through NLTK, run `nltk.download()` and follow the instructions to get the data. You'll probably want to select "`all`" or "`all corpora`", and then you'll be (mostly) set for the semester.

In [2]:
# Load the Brown corpus
sentences = list(nltk.corpus.brown.sents())
print "Loaded %d sentences (%g tokens)" % (len(sentences), sum(map(len, sentences)))

Loaded 57340 sentences (1.16119e+06 tokens)


In [3]:
def flatten(list_of_lists):
    return itertools.chain.from_iterable(list_of_lists)

Let's start by looking at the all the words in the corpus, and looking at some basic statistics.

We've built a helper class, `Vocabulary`, that ingests a list of words, counts their frequencies, and assigns each one a numerical ID that will be useful later on.

In [4]:
canonicalize = True  # if so, will make everything lowercase and canonicalize digits

if canonicalize:
    token_feed = (data_utils.canonicalize_word(w) for w in flatten(sentences))
else:
    token_feed = flatten(sentences)

vocab = vocabulary.Vocabulary(token_feed)
print "Vocabulary size: %d" % vocab.size

print "Most common unigrams:"
for word, count in vocab.unigram_counts.most_common(10):
    print "\"%s\": %d" % (word, count)

Vocabulary size: 48174
Most common unigrams:
"the": 69971
",": 58334
".": 49346
"of": 36412
"and": 28853
"to": 26158
"a": 23195
"in": 21337
"that": 10594
"is": 10109


`Vocabulary.unigram_counts` is a dictionary (actually, `collections.Counter`) of the unigram frequencies $c(w)$. Let's look at a plot of the top frequencies:

In [5]:
words, counts = zip(*vocab.unigram_counts.most_common(20))
data = [go.Bar(x=words, y=counts, name='Counts')]
plotly.iplot(data)

You'll notice that it falls off very quickly! Recall that word frequencies tend to follow **Zipf's law**, in that they are rougly proportional to $\frac{1}{\mathrm{rank}(w)}$, where rank = 1 for the most common word, 2 for the second-most, etc. We can test this directly with a numerical fit:

In [6]:
words, counts = zip(*vocab.unigram_counts.most_common(100))
counts = np.array(counts, dtype=float)
rank = 1 + np.arange(len(counts))
N = np.sum(counts)
p = counts / N

# Optimize least-squares in log space
# see http://nlp.stanford.edu/IR-book/html/htmledition/zipfs-law-modeling-the-distribution-of-terms-1.html
import scipy.optimize
fit_func = lambda (a, b): (np.log(a*p) - np.log(a * rank**b))
(a,b), _ = scipy.optimize.leastsq(fit_func, np.array([p[0], -1.0]))
print u"Power law exponent: \u03B2 = %.02f" % b
p_pred = (a * rank**b) / sum(a * rank**b)  # predict probabilities
c_pred = N * p_pred  # predict counts

# Plot counts, with fit curve
data = [go.Bar(x=words, y=counts, name='Counts'),
        go.Scatter(x=words, y=np.round(c_pred), name="Zipf Fit")]
layout=go.Layout(yaxis=dict(range=[0,1.2*max(counts)]))
fig = go.Figure(data=data, layout=layout)
plotly.iplot(fig)

Power law exponent: β = -1.41


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: RuntimeWarning:

invalid value encountered in log



## N-Gram Language Models

If we want, we can take our unigram counts and use them as a language model directly:
$$ P(w_i | w_{i-1}, ..., w_0) = P(w_i) $$

Here's a quick implementation. We don't need all the class machinery, but it's easier to see the structure that way:

In [7]:
class UnigramLM(object):
    def __init__(self, vocab):
        """Build the model."""
        self.words = vocab.word_to_id.keys()
        self.counts = np.array([vocab.unigram_counts[w] 
                                for w in self.words], dtype=float)
        self.probs = self.counts / sum(self.counts)
        # For looking up words when scoring
        self._word_to_idx = {w:i for i,w in enumerate(self.words)}
        
    def predict_next(self, seq):
        """Predict the next word in the sequence."""
        word = np.random.choice(self.words, p=self.probs)
        return word
    
    def score_seq(self, seq):
        """Score a sequence; returns estimated log probability."""
        score = 0.0
        for word in seq:
            idx = self._word_to_idx[word]
            score += np.log2(self.probs[idx])
        return score

Now we can generate a few sentences:

In [8]:
lm1 = UnigramLM(vocab)
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = []
    for i in range(max_length):
        seq.append(lm1.predict_next(seq))
        # Stop when we generate a period
        if seq[-1] == ".": break
    print " ".join(seq)
    print "[%d tokens; log P(seq): %.02f]" % (len(seq), lm1.score_seq(seq))
    print ""

came had my worked after `` scattered for or papa know with but they delight , philanthropic thum have normal
[20 tokens; log P(seq): -217.59]

of possible does with one's the i of `` sounds , the to of bargaining .
[16 tokens; log P(seq): -126.89]

unenvied as somewhat oriental base most mg. `` that .
[10 tokens; log P(seq): -115.44]

.
[1 tokens; log P(seq): -4.56]

case in out corpse on don top an i elders ? headline swells their dialogue a st. turned these maye
[20 tokens; log P(seq): -240.97]



**Bonus:** What is the *most likely* sequence that this unigram model could produce?

**Bonus:** How likely is this model to emit a period at each step? What is the expected length of a sentence? How are the lengths distributed?

### Bigram Model

The unigram model isn't very impressive - mostly, it just gives gibberish consisting of common words. Let's build a better model from bigrams:
$$ P(w_i | w_{i-1}, ..., w_0) = P(w_i | w_{i-1}) $$

**Caution!** We have a big vocabulary. How many possible bigrams are there? 

If we store it as a matrix $P_{ab} = P(w_i = b | w_{i-1} = a)$, how big a matrix will this be, in megabytes? (assume 8 bytes per entry)

In [9]:
#!pip install --upgrade psutil
import psutil
print "Needed:    %g MB" % (8 * (vocab.size ** 2) / (2**20))
print "Available: %g MB" % (psutil.virtual_memory().available / (2**20))

Needed:    17705 MB
Available: 424 MB


Well, that's no good. Even for a small vocabulary, we can't store all the possible bigrams.

**Q:** What if we consider all the possible bigrams *that exist in the corpus*? How many are there, at most?

Let's construct a language model that only stores observed bigrams. We'll have to do a little more work to deal with unobserved bigrams, but it'll be worth the memory savings.

We'll compute the probabilities by usual formula, starting with raw bigram counts:

$$  P_{ab} = P(w_i = b | w_{i-1} = a) = \frac{\mathrm{c(ab)}}{\sum_{a'}\mathrm{c(a'b)}} = \frac{C_{ab}}{\sum_{a'} C_{a'b}} $$

In [10]:
from collections import defaultdict

def normalize_counter(c):
    total = sum(c.itervalues())
    return {w:float(c[w])/total for w in c}

class SimpleBigramLM(object):
    def __init__(self, words):
        # We'll make this a defaultdict to simplify
        # adding new words as we see them.
        # counts will map {a: {b: count(a,b)}}
        self.bigram_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
        
        # Iterate through words, as a single pass
        # This way, we allow words to be a generator stream,
        # instead of needing it to all be in-memory.
        a_ = None
        for b_ in words:
            if a_ is not None:
                self.bigram_counts[a_][b_] += 1
            a_ = b_
            
        # Normalize to get P(b | a)
        self.bigram_probs = {a:normalize_counter(c) 
                             for a,c in self.bigram_counts.iteritems()}
    
    def predict_next(self, seq):
        if len(seq) == 0:
            raise ValueError("Simple Bigram LM needs context of at least 1 word!")
        # Get list of words, probs
        words, probs = zip(*self.bigram_probs[seq[-1]].items())
        return np.random.choice(words, p=probs)
    
    def score_seq(self, seq):
        # Score all bigrams
        score = 0.0
        for i in range(1, len(seq)):
            a,b = seq[i-1], seq[i]
            score += np.log2(self.bigram_probs[a][b])
        return score

We didn't do this for the unigram LM, but when modeling sentences it's helpful to add special beginning-of-sentence (`<s>`) and end-of-sentence (`</s>`) tokens. 

This lets the model estimate the probability of a word appearing at the beginning of a sentence, and lets it model the end of a sentence properly, since periods or other punctuation aren't always an accurate guide (e.g. `"Dr."` or `"Yahoo!"`).

Our padded sentences will look like this:
```
<s> the cat sat in the hat . </s>
```

In [11]:
padded_sentences = (["<s>"] + s + ["</s>"] for s in sentences)
token_feed = (data_utils.canonicalize_word(w) 
              for w in itertools.chain.from_iterable(padded_sentences))
t0 = time.time()
print "Building bigram LM...",
lm2 = SimpleBigramLM(token_feed)
print "done in %.02f s" % (time.time() - t0)

Building bigram LM... done in 2.64 s


In [12]:
unique_unigrams = len(lm2.bigram_probs)
unique_bigrams = sum(len(c) for c in lm2.bigram_probs.itervalues())
total_words = sum(sum(c.values()) for c in lm2.bigram_counts.itervalues())
print "Found %g bigrams in %g tokens (%g unique words)" % (unique_bigrams, total_words, unique_unigrams)

# Compute memory use, if we stored them in the most efficient way
# Assume 4 bytes for index a, 4 bytes for index b, 8 bytes for P_ab
# and 8 bytes of overhead (e.g. for pointers)
print "Optimal memory usage: %d MB" % ((unique_bigrams * 24) / (2**20))

Found 429128 bigrams in 1.27587e+06 tokens (48173 unique words)
Optimal memory usage: 9 MB


Now we can generate sentences, much as we did with the unigram LM. They should look a little better this time:

In [13]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>"]
    for i in range(max_length):
        seq.append(lm2.predict_next(seq))
        # Stop at end-of-sentence
        if seq[-1] == "</s>": break
    print " ".join(seq)
    print "[%d tokens; log P(seq): %.02f]" % (len(seq), lm2.score_seq(seq))
    print ""

<s> -- 16-mesh screening of newbury shore . </s>
[9 tokens; log P(seq): -41.86]

<s> another cure made evident than most fundamental law and instead we '' ? </s>
[15 tokens; log P(seq): -94.77]

<s> she would i asked tiredly repeat . </s>
[9 tokens; log P(seq): -38.46]

<s> well , fastened to the conductor '' . </s>
[10 tokens; log P(seq): -51.74]

<s> roberta said the music . </s>
[7 tokens; log P(seq): -34.10]



Looks much better than the unigram output! The generated sentences are a bit more grammatical, but still don't make much sense.

## Smoothing and Handling the Unknown

Our simple bigram model doesn't have any mechanism for handling unknown words - in fact, if we fed something unseen to `score_seq`, it would just crash:

In [14]:
lm2.score_seq(["<s>", "i", "love", "w266", "</s>"])

KeyError: 'w266'

We could just default to $ P(\texttt{<unk>} | w_{i-1}) = 0 $, but then we'd assign an implausibly low probability to anything we haven't seen before.

If we want to use our language model in the wild, we'll need to implement smoothing. Of course, there's lots of different ways to do this, and it's going to introduce hyperparameters we'll want to tune. So, we should also split off some of our data into a dev/validation set:

In [15]:
# Load a corpus.
# After you have things working, try changing "brown" to "state_union" or "gutenberg"
# You will also want to change the vocab size; 30k for brown, 10k for state_union, 20k for gutenberg.
# See http://www.nltk.org/nltk_data/ for a full list; you may need to use nltk.download() again.
sentences = list(nltk.corpus.brown.sents())
vocab_size = 30000
print "Loaded %d sentences (%g tokens)" % (len(sentences), sum(map(len, sentences)))

train_frac = 0.8
split_idx = int(train_frac * len(sentences))
train_sentences = sentences[:split_idx]
dev_sentences = sentences[split_idx:]

Loaded 57340 sentences (1.16119e+06 tokens)


Let's implement a smoothed trigram model. We'll build a single model that can do several different types of smoothing:

- Laplace / add-k smoothing
- Absolute discounting
- Backoff smoothing
- Interpolation
- Type fertility

The code for this is a bit lengthy, so we've moved it to **`ngram_lm.py`**.

In [16]:
# We'll use this to clip the vocabulary to the top words
token_feed = (data_utils.canonicalize_word(w) 
              for w in itertools.chain.from_iterable(sentences))
vocab_pruned = vocabulary.Vocabulary(token_feed, size=vocab_size)

padded_sentences = (["<s>", "<s>"] + s + ["</s>"] for s in train_sentences)
token_feed = (data_utils.canonicalize_word(w, wordset=vocab_pruned.word_to_id) 
              for w in itertools.chain.from_iterable(padded_sentences))

import ngram_lm
reload(ngram_lm)
t0 = time.time()
print "Building trigram LM...",
lm3 = ngram_lm.SmoothedTrigramLM(token_feed)
print "done in %.02f s" % (time.time() - t0)

Building trigram LM... done in 5.09 s


In [17]:
unique_unigrams = sum(len(c) for k,c in lm3.counts.iteritems()
                      if len(k) == 0)
unique_bigrams  = sum(len(c) for k,c in lm3.counts.iteritems()
                      if len(k) == 1)
unique_trigrams = sum(len(c) for k,c in lm3.counts.iteritems()
                      if len(k) == 2)
total_words = sum(sum(c.values()) for c in lm3.counts.itervalues())
print "Processed %g tokens. Found:" % total_words
print "  %g unique unigrams" % unique_unigrams
print "  %g unique bigrams" % unique_bigrams
print "  %g unique trigrams" % unique_trigrams

# Compute memory use, if we stored them in the most efficient way
# Assume 4 bytes for index a, 4 bytes for index b, 8 bytes for P_ab
# and 8 bytes of overhead (e.g. for pointers)
total_memory = (unique_unigrams * 20) + (unique_bigrams * 24) + (unique_trigrams * 28)
print "Optimal memory usage: %d MB" % (total_memory / (2**20))

Processed 3.35178e+06 tokens. Found:
  28706 unique unigrams
  356137 unique bigrams
  731783 unique trigrams
Optimal memory usage: 28 MB


Now we can generate sentences, as with the previous models! This will be a bit slow, as we don't precompute any of the smoothing calculations. *(We could be a lot faster by precomputing $ P(w_i | w_{i-1}, w_{i-2}) $ for all nonzero trigram counts and so on with the bigrams and unigrams, but this will do for now.)*

Change the parameters below to change the type of smoothing. The default is to use a KN model with $\delta = 0.75$, but you might also want to try un-commenting some of the others below.

**Q:** What do you notice about the per-token log probability on a smoothed model, versus an un-smoothed one? 

**Q:** Which model gives more plausible sentences? Do you expect that this model will generalize well?

In [18]:
# Set options here:
#   kn: true/false
#   delta: float
#   backoff: int (minimum count)
#   interpolation: (l1,l2,l3) (should sum to 1)
#   add_k: float
params = dict(kn=True, delta=0.75) # KN smoothing
# params = dict(kn=False, add_k=0.0) # unsmoothed!
# params = dict(kn=False, add_k=1.0) # add-1 (Laplace) smoothing
# params = dict(kn=False, add_k=1.0, interpolation=(0.2, 0.3, 0.5))
# params = dict(kn=False, add_k=1.0, backoff=5) # Katz smoothing

In [ ]:
max_length = 30
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]  # start with two to init trigram model
    for i in range(max_length):
        seq.append(lm3.predict_next(seq, **params))
        # Stop at end-of-sentence
        if seq[-1] == "</s>": break
    print " ".join(seq)
    score = lm3.score_seq(seq, **params)
    print "[%d tokens; log P(seq): %.02f, per-token: %.02f]" % (len(seq), score, 
                                                                score/(len(seq)-2))
    print ""

## Perplexity Measurement

Let's not forget the test set! The whole point of smoothing was do better on unseen data.

Which model performs best? Experiment with the parameters a bit to tune your LM.

**Q:** What happens if you use an unsmoothed model on this data?

In [ ]:
# params = dict(kn=True, delta=0.75) # KN smoothing
params = dict(kn=False, add_k=0.0) # unsmoothed!
# params = dict(kn=False, add_k=1.0) # add-1 (Laplace) smoothing
# params = dict(kn=False, add_k=1.0, interpolation=(0.2, 0.3, 0.5))
# params = dict(kn=False, add_k=1.0, backoff=5) # Katz smoothing

In [23]:
total_score = 0.0
total_tokens = 0

def preprocess_for_scoring(sentence):
    # Pre-process words, replace anything the model doesn't know
    # with <unk>
    words = [data_utils.canonicalize_word(w, wordset=known_words)
             for w in sentence]
    # Pad sequence with start and end markers
    return ["<s>", "<s>"] + words + ["</s>"]

known_words = set(lm3.words)
t0 = time.time()
for sentence in dev_sentences:
    seq = preprocess_for_scoring(sentence)
    score = lm3.score_seq(seq, **params)
    total_score += score
    total_tokens += (len(seq) - 2)
    
print "Scored dev set in %.02f s" % (time.time() - t0)
print "Average log score: %.02f" % (total_score / total_tokens)
print "Perplexity: %.02f" % (2**(-1*total_score/total_tokens))

Scored dev set in 1.73 s
Average log score: -inf
Perplexity: inf


## Linguistic Curiosities

You might have seen this floating around the internet last week:
![Adjective Order](adjective_order.jpg)
*source: https://twitter.com/MattAndersonBBC/status/772002757222002688?lang=en*

Let's see if it holds true, statistically at least. Note that log probabilities are always negative, so the smaller magnitude is better. And remember the log scale: a difference of score of 8 units means one utterance is $2^8 = 256$ times more likely!

In [24]:
s0 = preprocess_for_scoring("square green plastic toys".split())
s1 = preprocess_for_scoring("plastic green square toys".split())

params = dict(kn=True, delta=0.75)
print "s0 score: %.02f" % lm3.score_seq(s0, **params)
print "s1 score: %.02f" % lm3.score_seq(s1, **params)

s0 score: -62.12
s1 score: -70.78


We can also do a more scientific comparison, by looking at all possible permutations:

In [25]:
noun = "toys"
adjectives = ["square", "green", "plastic"]
results = []
for adjs in itertools.permutations(adjectives):
    words = list(adjs) + [noun]
    seq = preprocess_for_scoring(words)
    score = lm3.score_seq(seq, **params)
    results.append((score, words))

# Sort results
for score, words in sorted(results, reverse=True):
    print "\"%s\" : %.02f" % (" ".join(words), score)

"square green plastic toys" : -62.12
"green square plastic toys" : -62.15
"plastic square green toys" : -70.78
"plastic green square toys" : -70.78
"square plastic green toys" : -71.00
"green plastic square toys" : -71.03
